In [1]:
from math import sqrt
import pandas as pd
import numpy as np
import os
import torch
import torch.nn.functional as F
from rdkit import Chem
import matplotlib.pyplot as plt

from torch_geometric.loader import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import global_mean_pool, global_max_pool
%matplotlib inline

In [2]:
gpuid = 0
torch.cuda.set_device(gpuid)
print(torch.cuda.current_device())

0


In [3]:
import sys
sys.path.insert(0, '/home/was966/Research/bidd-clsar/')

In [4]:
from clsar.dataset import LSSNS, HSSMS
from clsar.feature import Gen39AtomFeatures
from clsar.model.model import ACANet_PNA, get_deg, _fix_reproducibility,ACANet_GAT, ACANet_GCN, ACANet_GIN
from clsar.model.loss import ACALoss, get_best_cliff
_fix_reproducibility(42)

In [5]:
ACANET_MODEL = ACANet_GCN

In [6]:
def train(train_loader, model, optimizer, aca_loss):

    total_examples = 0
    total_loss =  0    
    total_tsm_loss = 0
    total_reg_loss = 0   
    n_triplets = []
    n_pos_triplets = []
    model.train()
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        predictions, embeddings = model(data.x.float(), data.edge_index, 
                                        data.edge_attr, data.batch)
        
        loss_out = aca_loss(labels = data.y, 
                            predictions = predictions,
                            embeddings = embeddings)
        loss, reg_loss, tsm_loss, n, n_pos = loss_out
        
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
        total_tsm_loss += float(tsm_loss) * data.num_graphs        
        total_reg_loss += float(reg_loss) * data.num_graphs        
        total_examples += data.num_graphs
        
        n_triplets.append(int(n))
        n_pos_triplets.append(int(n_pos))
        
    train_loss = total_loss / total_examples
    total_tsm_loss = total_tsm_loss / total_examples
    total_reg_loss = total_reg_loss / total_examples
    n_triplets = int(sum(n_triplets) / (i+1))
    n_pos_triplets = int(sum(n_pos_triplets) / (i+1))
    
    return train_loss, total_tsm_loss, total_reg_loss, n_triplets, n_pos_triplets

@torch.no_grad()
def test(test_loader, model, aca_loss):
    model.eval()
    total_examples = 0
    total_loss = 0
    total_tsm_loss = 0
    total_reg_loss = 0
    n_triplets = []
    n_pos_triplets = []
    mse = []
    for i, data in enumerate(test_loader):
        data = data.to(device)
        predictions, embeddings = model(data.x.float(), data.edge_index,
                                        data.edge_attr, data.batch)
        loss_out = aca_loss(labels=data.y,
                            predictions=predictions,
                            embeddings=embeddings)

        loss, reg_loss, tsm_loss, n, n_pos = loss_out

        total_loss += float(loss) * data.num_graphs
        total_tsm_loss += float(tsm_loss) * data.num_graphs
        total_reg_loss += float(reg_loss) * data.num_graphs
        total_examples += data.num_graphs

        n_triplets.append(int(n))
        n_pos_triplets.append(int(n_pos))

        mse.append(F.mse_loss(predictions, data.y, reduction='none').cpu())

    test_loss = total_loss / total_examples
    total_tsm_loss = total_tsm_loss / total_examples
    total_reg_loss = total_reg_loss / total_examples
    n_triplets = int(sum(n_triplets) / (i+1))
    n_pos_triplets = int(sum(n_pos_triplets) / (i+1))
    
    test_rmse = float(torch.cat(mse, dim=0).mean().sqrt())
    
    return test_loss, total_tsm_loss, total_reg_loss, n_triplets, n_pos_triplets, test_rmse



def Test_performance(alpha=1.0):
    
    model = ACANET_MODEL(**pub_args, aggr = 'mean').to(device)  
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=10**-5)
    aca_loss = ACALoss(alpha=alpha, cliff_lower = 1., cliff_upper = 1.)
    
    history = []
    #ls_his = []
    for epoch in range(1, epochs):
        train_loss, tsm_loss, reg_loss, n_triplets, n_pos_triplets = train(train_loader, model, optimizer, aca_loss)

        _, _, _, _, train_n_pos_triplets, train_rmse = test(train_loader, model, aca_loss)
        _, _, _, _, val_n_pos_triplets, val_rmse = test(val_loader, model, aca_loss)
        _, _, _, _, test_n_pos_triplets, test_rmse = test(test_loader, model, aca_loss)

        
        print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f} tsm_loss: {tsm_loss:.4f} reg_loss: {reg_loss:.4f} '
              f'n_pos_triplets: {n_pos_triplets:03d};  Val: {val_rmse:.4f} Test: {test_rmse:.4f}')
    
        history.append({'Epoch':epoch, 'train_loss':train_loss, 'train_triplet_loss':tsm_loss,'train_mae_loss':reg_loss,
                        'val_rmse':val_rmse, 'test_rmse':test_rmse, 'train_rmse':train_rmse,'n_triplets': n_triplets, 
                        'n_pos_triplets':n_pos_triplets, 
                        'train_n_pos_triplets':train_n_pos_triplets,
                        'val_n_pos_triplets':val_n_pos_triplets,
                        'test_n_pos_triplets':test_n_pos_triplets,
                       
                       })
        #ls_his.append({'Epoch':epoch, 'mae_loss':float(mae_loss), 'triplet_loss':float(triplet_loss)})
    dfh = pd.DataFrame(history)
    return dfh

In [7]:
dataset_name = 'CHEMBL3979_EC50'
Dataset =  HSSMS #LSSNS 
epochs = 800
batch_size = 128
lr = 1e-4

pre_transform = Gen39AtomFeatures()
in_channels = pre_transform.in_channels
path = '../data/'

## model HPs
pub_args = {'in_channels':pre_transform.in_channels, 
            'edge_dim':pre_transform.edge_dim,
            'convs_layers': [64, 128, 256, 512],   
            'dense_layers': [256, 128, 32], 
            'out_channels':1, 
            # 'aggregators': ['mean', 'min', 'max', 'sum','std'],
            # 'scalers':['identity', 'amplification', 'attenuation'] ,
            'dropout_p': 0}

In [8]:
len(Dataset(path, name=dataset_name, pre_transform=pre_transform).shuffle(42))

1125

In [ ]:
# train, valid, test splitting
res1 = []
res2 = []
for seed in [8, 16, 24, 42, 64, 128, 256, 512, 1024, 2048]:
    dataset = Dataset(path, name=dataset_name, pre_transform=pre_transform).shuffle(42)
    N = len(dataset) // 5
    val_dataset = dataset[:N]
    test_dataset = dataset[N:2 * N]
    train_dataset = dataset[2 * N:]
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    deg = get_deg(train_dataset)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # With AC-Awareness ($\alpha = 1$)
    df1 = Test_performance(alpha=1.0)
    df1['seed'] = seed
    # Without AC-Awareness ($\alpha = 0$)
    df2 = Test_performance(alpha=0.0)
    df2['seed'] = seed
    res1.append(df1)
    res2.append(df2)

/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch_geometric/warnings.py:17: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch: 001, Loss: 11.2357 tsm_loss: 4.6409 reg_loss: 6.5948 n_pos_triplets: 226402;  Val: 6.0852 Test: 6.1176
Epoch: 002, Loss: 9.8413 tsm_loss: 3.9363 reg_loss: 5.9050 n_pos_triplets: 228764;  Val: 5.7904 Test: 5.8164
Epoch: 003, Loss: 8.5868 tsm_loss: 3.5585 reg_loss: 5.0283 n_pos_triplets: 228152;  Val: 5.1441 Test: 5.1747
Epoch: 004, Loss: 7.0034 tsm_loss: 3.1436 reg_loss: 3.8598 n_pos_triplets: 225069;  Val: 4.0747 Test: 4.1084
Epoch: 005, Loss: 5.1450 tsm_loss: 2.7784 reg_loss: 2.3667 n_pos_triplets: 225498;  Val: 2.4978 Test: 2.5298
Epoch: 006, Loss: 3.7488 tsm_loss: 2.6598 reg_loss: 1.0891 n_pos_triplets: 226579;  Val: 1.1534 Test: 1.1106
Epoch: 007, Loss: 3.7224 tsm_loss: 2.4534 reg_loss: 1.2690 n_pos_triplets: 221564;  Val: 1.0950 Test: 1.0300
Epoch: 008, Loss: 3.2741 tsm_loss: 2.2764 reg_loss: 0.9977 n_pos_triplets: 219174;  Val: 1.1472 Test: 1.1154
Epoch: 009, Loss: 3.0250 tsm_loss: 2.1026 reg_loss: 0.9224 n_pos_triplets: 217027;  Val: 1.2253 Test: 1.2039
Epoch: 010, Loss: 

/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch_geometric/warnings.py:17: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch: 001, Loss: 6.4871 tsm_loss: 5.0697 reg_loss: 6.4871 n_pos_triplets: 224117;  Val: 6.2231 Test: 6.2452
Epoch: 002, Loss: 5.8789 tsm_loss: 5.0111 reg_loss: 5.8789 n_pos_triplets: 223522;  Val: 5.9750 Test: 5.9953
Epoch: 003, Loss: 5.1107 tsm_loss: 4.8539 reg_loss: 5.1107 n_pos_triplets: 220625;  Val: 5.4398 Test: 5.4583
Epoch: 004, Loss: 4.0634 tsm_loss: 4.8849 reg_loss: 4.0634 n_pos_triplets: 222333;  Val: 4.4835 Test: 4.4960
Epoch: 005, Loss: 2.6064 tsm_loss: 4.9423 reg_loss: 2.6064 n_pos_triplets: 220328;  Val: 2.9674 Test: 2.9635
Epoch: 006, Loss: 1.1903 tsm_loss: 4.9643 reg_loss: 1.1903 n_pos_triplets: 217732;  Val: 1.2948 Test: 1.2200
Epoch: 007, Loss: 1.2456 tsm_loss: 4.7900 reg_loss: 1.2456 n_pos_triplets: 215482;  Val: 1.1043 Test: 1.0045
Epoch: 008, Loss: 1.0260 tsm_loss: 4.6517 reg_loss: 1.0260 n_pos_triplets: 214234;  Val: 1.2009 Test: 1.1330
Epoch: 009, Loss: 0.8835 tsm_loss: 4.4741 reg_loss: 0.8835 n_pos_triplets: 214189;  Val: 1.3029 Test: 1.2520
Epoch: 010, Loss: 0

/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch_geometric/warnings.py:17: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


...

Epoch: 001, Loss: 11.7051 tsm_loss: 5.1490 reg_loss: 6.5561 n_pos_triplets: 225006;  Val: 6.5488 Test: 6.4676
Epoch: 002, Loss: 10.3671 tsm_loss: 4.2196 reg_loss: 6.1475 n_pos_triplets: 225756;  Val: 6.3210 Test: 6.2432
Epoch: 003, Loss: 9.3097 tsm_loss: 3.6887 reg_loss: 5.6209 n_pos_triplets: 226299;  Val: 5.9079 Test: 5.8295
Epoch: 004, Loss: 8.3424 tsm_loss: 3.4807 reg_loss: 4.8617 n_pos_triplets: 227642;  Val: 5.1737 Test: 5.0991
Epoch: 005, Loss: 6.7464 tsm_loss: 2.9899 reg_loss: 3.7566 n_pos_triplets: 226288;  Val: 4.0416 Test: 3.9677
Epoch: 006, Loss: 5.1546 tsm_loss: 2.8368 reg_loss: 2.3178 n_pos_triplets: 225422;  Val: 2.4837 Test: 2.4054
Epoch: 007, Loss: 3.7845 tsm_loss: 2.6702 reg_loss: 1.1144 n_pos_triplets: 225769;  Val: 1.2153 Test: 1.1502
Epoch: 008, Loss: 3.6699 tsm_loss: 2.4430 reg_loss: 1.2269 n_pos_triplets: 223065;  Val: 1.1555 Test: 1.1161
Epoch: 009, Loss: 3.3084 tsm_loss: 2.2486 reg_loss: 1.0598 n_pos_triplets: 223345;  Val: 1.1684 Test: 1.1147
Epoch: 010, Loss:

/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch_geometric/warnings.py:17: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch: 001, Loss: 6.5766 tsm_loss: 5.3869 reg_loss: 6.5766 n_pos_triplets: 222367;  Val: 6.5144 Test: 6.4366
Epoch: 002, Loss: 6.1319 tsm_loss: 5.2395 reg_loss: 6.1319 n_pos_triplets: 222609;  Val: 6.3875 Test: 6.3082
Epoch: 003, Loss: 5.5001 tsm_loss: 5.0614 reg_loss: 5.5001 n_pos_triplets: 219329;  Val: 5.9100 Test: 5.8301
Epoch: 004, Loss: 4.5696 tsm_loss: 4.9842 reg_loss: 4.5696 n_pos_triplets: 220294;  Val: 5.0461 Test: 4.9642
Epoch: 005, Loss: 3.2513 tsm_loss: 4.9388 reg_loss: 3.2513 n_pos_triplets: 217367;  Val: 3.6554 Test: 3.5741
Epoch: 006, Loss: 1.6029 tsm_loss: 4.8084 reg_loss: 1.6029 n_pos_triplets: 218035;  Val: 1.7798 Test: 1.6963
Epoch: 007, Loss: 1.1767 tsm_loss: 4.7411 reg_loss: 1.1767 n_pos_triplets: 214382;  Val: 1.1834 Test: 1.1436
Epoch: 008, Loss: 1.2421 tsm_loss: 4.6569 reg_loss: 1.2421 n_pos_triplets: 213998;  Val: 1.1389 Test: 1.1023
Epoch: 009, Loss: 0.9214 tsm_loss: 4.4734 reg_loss: 0.9214 n_pos_triplets: 212846;  Val: 1.3619 Test: 1.3289
Epoch: 010, Loss: 0

/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/home/was966/micromamba/envs/responder/lib/python3.8/site-packages/torch_geometric/warnings.py:17: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)


Epoch: 001, Loss: 12.8495 tsm_loss: 6.3775 reg_loss: 6.4721 n_pos_triplets: 224026;  Val: 6.2729 Test: 6.2118
Epoch: 002, Loss: 11.5675 tsm_loss: 5.5044 reg_loss: 6.0630 n_pos_triplets: 224220;  Val: 6.0586 Test: 6.0058
Epoch: 003, Loss: 10.1684 tsm_loss: 4.6325 reg_loss: 5.5359 n_pos_triplets: 221222;  Val: 5.6473 Test: 5.6058
Epoch: 004, Loss: 8.8065 tsm_loss: 3.9978 reg_loss: 4.8087 n_pos_triplets: 221443;  Val: 4.9658 Test: 4.9355
Epoch: 005, Loss: 7.4154 tsm_loss: 3.5926 reg_loss: 3.8229 n_pos_triplets: 218217;  Val: 3.9417 Test: 3.9289
Epoch: 006, Loss: 5.6833 tsm_loss: 3.1567 reg_loss: 2.5266 n_pos_triplets: 216883;  Val: 2.5626 Test: 2.5741
Epoch: 007, Loss: 4.2184 tsm_loss: 2.9771 reg_loss: 1.2413 n_pos_triplets: 215826;  Val: 1.2826 Test: 1.3291
Epoch: 008, Loss: 3.8739 tsm_loss: 2.7278 reg_loss: 1.1461 n_pos_triplets: 215707;  Val: 1.1870 Test: 1.2217
Epoch: 009, Loss: 3.6153 tsm_loss: 2.4863 reg_loss: 1.1290 n_pos_triplets: 212766;  Val: 1.1435 Test: 1.1659
Epoch: 010, Loss

In [ ]:
df1 = pd.concat(res1)
df2 = pd.concat(res2)

In [ ]:
df1.to_csv('./with_aca.csv')
df2.to_csv('./without_aca.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
colors = ['#FFE699','#00B0F0']


y = 'val_rmse'

n1 = r'With AC-awareness ($\mathcal{L}_{mae} + \mathcal{L}_{tsm}$)'
n2 = r'Without AC-awareness ($\mathcal{L}_{mae}$)'


dfp = df2.groupby('Epoch').mean()[y].to_frame(name = n2).join(df1.groupby('Epoch').mean()[y].to_frame(name = n1)).rolling(1).mean()
dfp_std = df2.groupby('Epoch').std()[y].to_frame(name = n2).join(df1.groupby('Epoch').std()[y].to_frame(name = n1)).rolling(1).mean()

dfp.plot(lw = 2, ax=ax,color = colors, alpha =1)
ax.fill_between(dfp.index, (dfp - dfp_std)[n1], (dfp + dfp_std)[n1], color=colors[1], alpha=0.2)
ax.fill_between(dfp.index, (dfp - dfp_std)[n2], (dfp + dfp_std)[n2], color=colors[0], alpha=0.2)

ax.set_ylim(0.60, 1.0)
ax.set_ylabel('Validation RMSE')
ax.set_xlabel('epochs')
ax.spines[['right', 'top']].set_visible(False)

ax.set_xlim(1,800)

ax.tick_params(left='off', labelleft='on', labelbottom='on', bottom = 'off',  pad=.5,)
fig.savefig('./Validation_RMSE.svg', bbox_inches='tight', dpi=400) 
fig.savefig('./Validation_RMSE.pdf', bbox_inches='tight', dpi=400) 

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
colors = ['#FFE699','#00B0F0']


y = 'test_rmse'



dfp = df2.groupby('Epoch').mean()[y].to_frame(name = n2).join(df1.groupby('Epoch').mean()[y].to_frame(name = n1)).rolling(1).mean()
dfp_std = df2.groupby('Epoch').std()[y].to_frame(name = n2).join(df1.groupby('Epoch').std()[y].to_frame(name = n1)).rolling(1).mean()

dfp.plot(lw = 2, ax=ax,color = colors, alpha =1)
ax.fill_between(dfp.index, (dfp - dfp_std)[n1], (dfp + dfp_std)[n1], color=colors[1], alpha=0.2)
ax.fill_between(dfp.index, (dfp - dfp_std)[n2], (dfp + dfp_std)[n2], color=colors[0], alpha=0.2)

ax.set_ylim(0.60, 1.0)
ax.set_ylabel('Test RMSE')
ax.set_xlabel('epochs')
ax.spines[['right', 'top']].set_visible(False)

ax.set_xlim(1,800)

ax.tick_params(left='off', labelleft='on', labelbottom='on', bottom = 'off',  pad=.5,)
fig.savefig('./Test_RMSE.svg' , bbox_inches='tight', dpi=400) 
fig.savefig('./Test_RMSE.pdf' , bbox_inches='tight', dpi=400) 

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))


y = 'n_pos_triplets'

dfp = df2.groupby('Epoch').mean()[y].to_frame(name = n2).join(df1.groupby('Epoch').mean()[y].to_frame(name = n1)).rolling(1).mean()
dfp_std = df2.groupby('Epoch').std()[y].to_frame(name = n2).join(df1.groupby('Epoch').std()[y].to_frame(name = n1)).rolling(1).mean()

dfp.plot(lw = 3, ax=ax,color = colors, alpha =1)
ax.fill_between(dfp.index, (dfp - dfp_std)[n1], (dfp + dfp_std)[n1], color=colors[1], alpha=0.3)
ax.fill_between(dfp.index, (dfp - dfp_std)[n2], (dfp + dfp_std)[n2], color=colors[0], alpha=0.3)

ax.legend(loc='center', bbox_to_anchor=(0.55, 0.5))

ax.spines[['right', 'top']].set_visible(False)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax.set_ylabel("No. of HV-ACTs ($M^'$)")
ax.set_xlabel('epochs')
ax.tick_params(left='off', labelleft='on', labelbottom='on', bottom = 'off',  pad=.5,)
ax.set_xlim(-5,800)


fig.savefig('./Number_of_mined_ACTs_during_training.svg' , bbox_inches='tight', dpi=400) 
fig.savefig('./Number_of_mined_ACTs_during_training.pdf' , bbox_inches='tight', dpi=400) 

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
y = 'train_triplet_loss'
dfp = df2.groupby('Epoch').mean()[y].to_frame(name = n2).join(df1.groupby('Epoch').mean()[y].to_frame(name = n1)).rolling(1).mean()
dfp_std = df2.groupby('Epoch').std()[y].to_frame(name = n2).join(df1.groupby('Epoch').std()[y].to_frame(name = n1)).rolling(1).mean()

dfp.plot(lw = 3, ax=ax,color = colors, alpha =1)
ax.fill_between(dfp.index, (dfp - dfp_std)[n1], (dfp + dfp_std)[n1], color=colors[1], alpha=0.3)
ax.fill_between(dfp.index, (dfp - dfp_std)[n2], (dfp + dfp_std)[n2], color=colors[0], alpha=0.3)

ax.spines[['right', 'top']].set_visible(False)
ax.set_xlim(-5,800)
ax.set_ylim(-1,10)

ax.set_ylabel('Training TSM Loss')
ax.set_xlabel('epochs')
ax.tick_params(left='off', labelleft='on', labelbottom='on', bottom = 'off',  pad=.5,)
fig.savefig('./Triplet_loss_during_training.svg', bbox_inches='tight', dpi=400) 
fig.savefig('./Triplet_loss_during_training.pdf', bbox_inches='tight', dpi=400) 

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))

y = 'train_mae_loss'

dfp = df2.groupby('Epoch').mean()[y].to_frame(name = n2).join(df1.groupby('Epoch').mean()[y].to_frame(name = n1)).rolling(1).mean()
dfp_std = df2.groupby('Epoch').std()[y].to_frame(name = n2).join(df1.groupby('Epoch').std()[y].to_frame(name = n1)).rolling(1).mean()

dfp.plot(lw = 2.5, ax=ax,color = colors, alpha =1)
ax.fill_between(dfp.index, (dfp - dfp_std)[n1], (dfp + dfp_std)[n1], color=colors[1], alpha=0.3)
ax.fill_between(dfp.index, (dfp - dfp_std)[n2], (dfp + dfp_std)[n2], color=colors[0], alpha=0.3)



ax.set_ylim(0.0, 0.8)
ax.spines[['right', 'top']].set_visible(False)


ax.set_ylabel('Training MAE loss')
ax.set_xlabel('epochs')
ax.legend(loc='center', bbox_to_anchor=(0.55, 0.5))

ax.set_xlim(1,800)

ax.tick_params(left='off', labelleft='on', labelbottom='on', bottom = 'off',  pad=.5,)
fig.savefig('./Train_mae_los.svg', bbox_inches='tight', dpi=400) 
fig.savefig('./Train_mae_los.pdf', bbox_inches='tight', dpi=400) 